# Empirical calculation of the risk

This notebook contain the experiment described in Section 5.2 of the paper:

> In Section 4.2 we derived an upper bound on $Var(D)$. To help un-derstand the risk in acquiring E&M capabilities, here we performan empirical calculation on $Var(D)$ to determine how far we arefrom the bound in general.

In [15]:
from normal_normal_model import var_V, get_prioritisation_value_samples
from util import get_bootstrap_var
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
def print_run_setting(count, N, M, mu_X, sigma_X, mu_eps, sigma_1, sigma_2):
    print("Run {}: N = {}, M = {}, mu_X = {}, sigma_X = {}, "
          "mu_epsilon = {}, sigma_1 = {}, sigma_2 = {}"
          .format(count, N, M, np.round(mu_X, 4), 
                  np.round(sigma_X, 4), np.round(mu_eps, 4),
                  np.round(sigma_1, 4), np.round(sigma_2, 4)))

def calculate_var_and_bound(runs=500):
    
    # Constants
    NUM_SAMPLES = 1000
    NUM_BOOTSTRAPS = 500
    run_counter = 0
    
    rel_differences = []
    improvement_bootstrap_vars = []
    improvement_theoretical_var_bounds = []

    for run in range(1, runs+1, 1):
        N = int(10 ** np.random.uniform(1, 3.5))
        M = int(N * np.random.uniform(0.02, 0.6)) + 2
        sigma_X = np.sqrt(np.random.chisquare(3))
        sigma_1 = np.sqrt(np.random.chisquare(3))
        sigma_2 = sigma_1 * np.random.uniform(0.02, 0.9999)
        mu_X = np.random.normal(0, 3)
        mu_epsilon = np.random.normal(0, 3)

        # Reconciling the use of sigmas by numpy
        # and sigma_sqs in the theoretical calculations
        sigma_sq_1 = sigma_1 ** 2
        sigma_sq_2 = sigma_2 ** 2
        sigma_sq_X = sigma_X ** 2

        print_run_setting(run, N, M, mu_X, sigma_X, 
                          mu_epsilon, sigma_1, sigma_2)

        _, _, improvement = (
            get_prioritisation_value_samples(
                NUM_SAMPLES, N, M, mu_X, mu_epsilon, 
                sigma_sq_X, sigma_sq_1, sigma_sq_2, verbose=False)
        )

        improvement_bootstrap_var = (
            np.mean(get_bootstrap_var(improvement, NUM_BOOTSTRAPS))
        )
        improvement_theoretical_var_bound = (
            var_V(sigma_sq_X, sigma_sq_1, N, M) +
            var_V(sigma_sq_X, sigma_sq_2, N, M)
        )
        rel_difference = improvement_bootstrap_var / improvement_theoretical_var_bound
        
        improvement_bootstrap_vars.append(improvement_bootstrap_var)
        improvement_theoretical_var_bounds.append(improvement_theoretical_var_bound)
        rel_differences.append(rel_difference)
        
        print("Run {}: Mean bootstrap Var(D) = {} (High bound: {})"
              .format(run,
                      improvement_bootstrap_var, 
                      improvement_theoretical_var_bound))
    
        if (run % 20 == 0):
            print("Average var to bound ratio so far: {}"
                  .format(np.mean(rel_differences)))
            
    return improvement_bootstrap_vars, improvement_theoretical_var_bounds, rel_differences

In [3]:
rel_differences = []
improvement_bootstrap_vars = []
improvement_theoretical_var_bounds = []

# Get the bootstrap variance and theoretical high bound in batches of ten
# So that if we require to abort the process we can still access
# some of the results we are interested in.
while True:
    variances, bounds, diffs = calculate_var_and_bound(10)
    improvement_bootstrap_vars = improvement_bootstrap_vars + variances
    improvement_theoretical_var_bounds = improvement_theoretical_var_bounds + bounds
    rel_differences = rel_differences + diffs

Run 1: N = 82, M = 43, mu_X = 1.3005, sigma_X = 1.0937, mu_epsilon = 0.2135, sigma_1 = 2.3993, sigma_2 = 1.1379
Run 1: Mean bootstrap Var(D) = 0.014040348036837659 (High bound: 0.04981849760386278)
Run 2: N = 179, M = 16, mu_X = 1.4591, sigma_X = 1.3842, mu_epsilon = 1.0635, sigma_1 = 1.2019, sigma_2 = 0.2203
Run 2: Mean bootstrap Var(D) = 0.031652725550030604 (High bound: 0.11744265630127124)
Run 3: N = 1285, M = 632, mu_X = -2.7363, sigma_X = 1.8716, mu_epsilon = -1.7873, sigma_1 = 0.8534, sigma_2 = 0.8289
Run 3: Mean bootstrap Var(D) = 0.0002941623148218195 (High bound: 0.008089429529369502)
Run 4: N = 2044, M = 885, mu_X = -3.5366, sigma_X = 1.3999, mu_epsilon = -0.6112, sigma_1 = 0.514, sigma_2 = 0.4149
Run 4: Mean bootstrap Var(D) = 6.26464134486189e-05 (High bound: 0.002983556589682002)
Run 5: N = 19, M = 6, mu_X = -3.2579, sigma_X = 1.8719, mu_epsilon = 0.2911, sigma_1 = 2.4697, sigma_2 = 2.3725
Run 5: Mean bootstrap Var(D) = 0.3582418820352323 (High bound: 0.9459985819516208)


Run 2: Mean bootstrap Var(D) = 0.007091824467301356 (High bound: 0.05822563276444885)
Run 3: N = 2043, M = 208, mu_X = -0.4349, sigma_X = 1.09, mu_epsilon = -6.8249, sigma_1 = 1.1022, sigma_2 = 1.0695
Run 3: Mean bootstrap Var(D) = 0.0037758036240924733 (High bound: 0.007816886874017905)
Run 4: N = 12, M = 4, mu_X = 1.0797, sigma_X = 1.4177, mu_epsilon = 0.8624, sigma_1 = 0.6648, sigma_2 = 0.5251
Run 4: Mean bootstrap Var(D) = 0.034924868825442185 (High bound: 0.5595672397971156)
Run 5: N = 1292, M = 110, mu_X = 0.8552, sigma_X = 2.6814, mu_epsilon = -3.3879, sigma_1 = 1.637, sigma_2 = 1.173
Run 5: Mean bootstrap Var(D) = 0.012172311236443346 (High bound: 0.06409866117092262)
Run 6: N = 20, M = 10, mu_X = -1.2264, sigma_X = 2.6734, mu_epsilon = -0.5743, sigma_1 = 1.6122, sigma_2 = 1.4044
Run 6: Mean bootstrap Var(D) = 0.06651277594398736 (High bound: 1.0170489154057782)
Run 7: N = 21, M = 5, mu_X = 1.6954, sigma_X = 0.8242, mu_epsilon = 3.311, sigma_1 = 0.8223, sigma_2 = 0.314
Run 7: M

Run 4: Mean bootstrap Var(D) = 0.0009779271125660333 (High bound: 0.003254170776767641)
Run 5: N = 17, M = 12, mu_X = -0.4615, sigma_X = 2.7595, mu_epsilon = 4.4242, sigma_1 = 1.5412, sigma_2 = 0.5609
Run 5: Mean bootstrap Var(D) = 0.02031373153855537 (High bound: 0.9789201323238446)
Run 6: N = 519, M = 193, mu_X = -1.3905, sigma_X = 1.6178, mu_epsilon = -1.029, sigma_1 = 0.6888, sigma_2 = 0.5537
Run 6: Mean bootstrap Var(D) = 0.0006300350646115056 (High bound: 0.017532796149012785)
Run 7: N = 477, M = 166, mu_X = 1.1258, sigma_X = 1.0929, mu_epsilon = 1.8151, sigma_1 = 1.4614, sigma_2 = 0.5069
Run 7: Mean bootstrap Var(D) = 0.002652443242493771 (High bound: 0.010795525705803131)
Run 8: N = 1768, M = 965, mu_X = 0.1474, sigma_X = 2.1059, mu_epsilon = 1.4005, sigma_1 = 2.1778, sigma_2 = 1.2574
Run 8: Mean bootstrap Var(D) = 0.0008747941291737027 (High bound: 0.007563436029818026)
Run 9: N = 23, M = 7, mu_X = -2.7678, sigma_X = 1.3406, mu_epsilon = 1.4692, sigma_1 = 1.4995, sigma_2 = 0.4

Run 5: Mean bootstrap Var(D) = 0.0030250094792331453 (High bound: 0.013444349205798017)
Run 6: N = 60, M = 28, mu_X = -3.1644, sigma_X = 2.2883, mu_epsilon = -1.2341, sigma_1 = 1.98, sigma_2 = 1.8625
Run 6: Mean bootstrap Var(D) = 0.044482504542879095 (High bound: 0.29427157256544345)
Run 7: N = 86, M = 41, mu_X = -2.3768, sigma_X = 0.4286, mu_epsilon = 4.1789, sigma_1 = 1.3348, sigma_2 = 1.1549
Run 7: Mean bootstrap Var(D) = 0.003922489379373287 (High bound: 0.008624412909050071)
Run 8: N = 266, M = 86, mu_X = 1.2012, sigma_X = 2.0501, mu_epsilon = 0.8101, sigma_1 = 1.0186, sigma_2 = 0.2621
Run 8: Mean bootstrap Var(D) = 0.0024167022601025248 (High bound: 0.05906212222781895)
Run 9: N = 261, M = 62, mu_X = 1.8464, sigma_X = 1.5566, mu_epsilon = -0.7596, sigma_1 = 0.2905, sigma_2 = 0.228
Run 9: Mean bootstrap Var(D) = 0.00023180590553674007 (High bound: 0.03948471108588113)
Run 10: N = 11, M = 7, mu_X = 1.8605, sigma_X = 0.2259, mu_epsilon = -3.6937, sigma_1 = 1.6745, sigma_2 = 1.2097


Run 6: Mean bootstrap Var(D) = 0.015066565105492365 (High bound: 0.2893884583666153)
Run 7: N = 48, M = 11, mu_X = 0.2029, sigma_X = 1.6001, mu_epsilon = 4.8554, sigma_1 = 1.8381, sigma_2 = 1.4039
Run 7: Mean bootstrap Var(D) = 0.1143832931247625 (High bound: 0.33990806911854915)
Run 8: N = 1031, M = 515, mu_X = -3.1328, sigma_X = 1.3677, mu_epsilon = 2.33, sigma_1 = 1.6761, sigma_2 = 0.8012
Run 8: Mean bootstrap Var(D) = 0.0009313596193469604 (High bound: 0.0059336479908594685)
Run 9: N = 332, M = 36, mu_X = 0.4603, sigma_X = 0.8799, mu_epsilon = -3.1982, sigma_1 = 0.9659, sigma_2 = 0.9035
Run 9: Mean bootstrap Var(D) = 0.015016913997910301 (High bound: 0.03029361356966795)
Run 10: N = 60, M = 19, mu_X = 4.9559, sigma_X = 1.2112, mu_epsilon = -0.4209, sigma_1 = 1.8441, sigma_2 = 1.1637
Run 10: Mean bootstrap Var(D) = 0.04560225316393816 (High bound: 0.1248193440390756)
Run 1: N = 429, M = 199, mu_X = -3.1989, sigma_X = 2.4372, mu_epsilon = 1.5773, sigma_1 = 2.088, sigma_2 = 0.8054
Run

Run 7: Mean bootstrap Var(D) = 0.004774180897510751 (High bound: 0.05224561619577213)
Run 8: N = 54, M = 13, mu_X = -5.1455, sigma_X = 1.0919, mu_epsilon = 3.6611, sigma_1 = 1.0628, sigma_2 = 0.2777
Run 8: Mean bootstrap Var(D) = 0.022938471954142715 (High bound: 0.1129700751042492)
Run 9: N = 51, M = 22, mu_X = 1.7848, sigma_X = 1.4199, mu_epsilon = -2.2268, sigma_1 = 1.3741, sigma_2 = 0.5061
Run 9: Mean bootstrap Var(D) = 0.017988762740445512 (High bound: 0.13306055275523343)
Run 10: N = 2464, M = 114, mu_X = 3.8874, sigma_X = 2.9928, mu_epsilon = -6.0367, sigma_1 = 1.4321, sigma_2 = 0.9334
Run 10: Mean bootstrap Var(D) = 0.008964588615571586 (High bound: 0.06156793099965755)
Run 1: N = 1181, M = 709, mu_X = -7.5567, sigma_X = 1.4235, mu_epsilon = 1.4842, sigma_1 = 0.6752, sigma_2 = 0.5618
Run 1: Mean bootstrap Var(D) = 0.00011155739404543715 (High bound: 0.004481416929144096)
Run 2: N = 81, M = 49, mu_X = -1.369, sigma_X = 3.2209, mu_epsilon = 0.9287, sigma_1 = 0.8622, sigma_2 = 0.0

Run 9: Mean bootstrap Var(D) = 0.00026043891725427377 (High bound: 0.0050584537350938956)
Run 10: N = 398, M = 171, mu_X = 1.5572, sigma_X = 1.687, mu_epsilon = 2.0681, sigma_1 = 1.3772, sigma_2 = 0.0516
Run 10: Mean bootstrap Var(D) = 0.0021225945017904406 (High bound: 0.023494931464733505)
Run 1: N = 367, M = 24, mu_X = -2.688, sigma_X = 1.659, mu_epsilon = 2.994, sigma_1 = 1.9011, sigma_2 = 1.3279
Run 1: Mean bootstrap Var(D) = 0.07536343082977268 (High bound: 0.14754653863290906)
Run 2: N = 2242, M = 710, mu_X = 1.5791, sigma_X = 2.0506, mu_epsilon = -3.6142, sigma_1 = 2.4271, sigma_2 = 1.6666
Run 2: Mean bootstrap Var(D) = 0.002675575014730377 (High bound: 0.00917610485987548)
Run 3: N = 306, M = 57, mu_X = 7.1177, sigma_X = 3.4115, mu_epsilon = -4.0914, sigma_1 = 1.3902, sigma_2 = 1.2665
Run 3: Mean bootstrap Var(D) = 0.01458574690001254 (High bound: 0.21290172187797324)
Run 4: N = 17, M = 4, mu_X = 2.0712, sigma_X = 1.5395, mu_epsilon = 4.3817, sigma_1 = 1.2131, sigma_2 = 0.5999

KeyboardInterrupt: 

In [38]:
plt.clf()
plt.figure(figsize=(4, 2))
plt.hist(rel_differences, bins=20)
plt.xlim(0.00, 1.01)
plt.xlabel('Empirical variance / Theoretical bound')
plt.ylabel('Count')
# plt.show()
plt.savefig('../output/ratio_empirical_bound_distribution.pdf',
            bbox_inches='tight')